In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import json
import time
import datetime as dt
import re

In [2]:
# Dealers description
dealer_dct = {
    'id': 'khushiauto',
    'name': 'Khushi Auto',
    'street': '2783 Derry Rd E.',
    'city': 'Mississauga',
    'zip': 'L4T1A3',
    'province': 'ON',
    'phone': '905-460-9624',
    'url': 'https://www.khushiauto.ca',
    'latitude': 43.700880,
    'longitude': -79.646490,
}

print(json.dumps(dealer_dct, indent=4))

with open('data/khushiauto.json', 'w', encoding='utf-8') as fp:
    fp.writelines(json.dumps(dealer_dct) + "\n")

{
    "id": "khushiauto",
    "name": "Khushi Auto",
    "street": "2783 Derry Rd E.",
    "city": "Mississauga",
    "zip": "L4T1A3",
    "province": "ON",
    "phone": "905-460-9624",
    "url": "https://www.khushiauto.ca",
    "latitude": 43.70088,
    "longitude": -79.64649
}


In [3]:
base_url ='https://www.khushiauto.ca'
inventory_url = 'https://www.khushiauto.ca/used-cars'

In [4]:
# Function uses inventory url (defined above), takes url parameters (such as a page number) as a dict,
# parses the inventory page and returns a list of URLs to car description pages
# Parsing stops when first sold vehicle found or when no search results found on a page
# The function returns a list or URLs and a boolean: True if there might be more data, False when done
def parse_inventory(query_args):
    global base_url, inventory_url
    res = []

    time.sleep(1.1)
    response = requests.get(inventory_url, query_args)
    
    soup = BeautifulSoup(response.text, 'lxml')
    vehicles = soup.find_all('div', class_='vehicle search-result-item vehicleList')
    if vehicles is None:
        return res, False
    
    for v in vehicles:
        # check if it isn't sold
        sold = v.find('div', class_='SoldVehicle')
        if sold is not None:
            return res, False # all the rest of the inventory is a list of sold vehicles, no need to go further

        # get vehicle description page url
        url = base_url + v.div.contents[1].div.div.a['href']
        
        # Printing URL (for debugginh purposes)
        print(url)
        res.append(url)
    
    return res, True

In [5]:
# Making a list of inventory URLs
vehicle_urls = []
query_args = {'ppage': 100, 'cpage': 1}

urls, has_more = parse_inventory(query_args)

while has_more:
    vehicle_urls.extend(urls)
    query_args['cpage'] += 1
    urls, has_more = parse_inventory(query_args)
    
vehicle_urls.extend(urls)
print(len(vehicle_urls))

https://www.khushiauto.ca/cars/used/2016-Nissan-Altima-Sedan-114128
https://www.khushiauto.ca/cars/used/2017-Hyundai-Elantra-Sedan-114118
https://www.khushiauto.ca/cars/used/2012-Mercedes-Benz-CL-Class-Coupe-113364
https://www.khushiauto.ca/cars/used/2015-Acura-ILX-Sedan-113362
https://www.khushiauto.ca/cars/used/2010-Nissan-Altima-Coupe-111392
https://www.khushiauto.ca/cars/used/2008-Nissan-Rogue-SUV-75061
https://www.khushiauto.ca/cars/used/2014-Audi-Q5-SUV-97354
https://www.khushiauto.ca/cars/used/2014-BMW-3-Series-Sedan-75072
https://www.khushiauto.ca/cars/used/2012-Mercedes-Benz-C-Class-Coupe-101233
https://www.khushiauto.ca/cars/used/2014-BMW-3-Series-Sedan-108795
https://www.khushiauto.ca/cars/used/2014-Chevrolet-Cruze-Sedan-103052
https://www.khushiauto.ca/cars/used/2014-Nissan-Pathfinder-SUV-104208
https://www.khushiauto.ca/cars/used/2011-Land-Rover-Range-Rover-SUV-100807
https://www.khushiauto.ca/cars/used/2015-Honda-Civic-Sedan-101920
https://www.khushiauto.ca/cars/used/2015

In [6]:
# Function parses vehicle description page and returns a dictionary with its parameters
def parse_car_infopage(url):
    # Translation table from their names to our key names
    translation_table = {
        'Make:': 'make',
        'Model:': 'model',
        'Year:': 'year',
        'Body Style:': 'body_style',
        'Odometer:': 'mileage', # Note: special treatment
        'Transmission:': 'transmission',
        'Engine:': 'cylinders', # Note: special treatment
        'Engine Size:': 'displacement',
        'Driveline:': 'drivetrain',
        'Exterior Color:': 'ext_color', 
        'Interior Color:': 'int_color',
        'Doors:': 'doors', # Note: special treatment
        'Passengers:': 'passengers',
        'Stock Number:': 'stock_id',
        'VIN:': 'vin'
    }
    
    # Fetching the page
    time.sleep(1)
    response = requests.get(url)

    # Parsing
    soup = BeautifulSoup(response.text, 'lxml')
    price_node = soup.find('span', class_='PriceValue')
    details_node = soup.find('div', class_='VehicleInfoDetails')
    descr_node = soup.find('div', class_='seller_comments')
    features_node = soup.find('ul', class_='VehicleOptions')

    # Basic info
    res = {
        'timestamp': dt.datetime.now().isoformat(),
        'dealer': 'khushiauto',
        'url': url
    }

    # Price
    try:
        price = re.sub('[^0-9]', '', price_node.contents[0]) # Cleaning junk, leaving only numbers
        res['price'] = int(price)
    except:
        res['invalid'] = 'invalid'
        return res

    # Reading core parameters
    spans = details_node.find_all('span')

    for s in spans:
        if s.text in translation_table:
            key = translation_table[s.text]
            value = s.next_sibling.text

            # Ad Hoc treatment
            if key in ['cylinders', 'doors']:
                try:
                    value = int(value.split()[0])
                except:
                    key = ''
            elif key == 'mileage':
                try:
                    value = int(re.sub('[^0-9]', '', value)) # Remove all non numbers using regexp substitution
                except:
                    key = ''
            elif key in ['year', 'passengers']:
                try:
                    value = int(value)
                except:
                    key = ''
            elif key in ['displacement']:
                try:
                    value = float(value)
                except:
                    key = ''

            if key:
                res[key] = value
    
    # Free text description - skip it, it's full of junk
    # res['description'] = descr_node.find('p').text

    # Features and options
    res['features'] = []
    items = features_node.find_all('li')
    for i in items:
        res['features'].append(i.text)
    
    # Printing result (for debugging purposes)
    print(json.dumps(res, indent=4))
    
    # Done
    return res

In [7]:
# Now scraping vehicle data, one by one
vehicle_data = []

for url in vehicle_urls:
    res = parse_car_infopage(url)
    if 'invalid' not in res:
        vehicle_data.append(res)
    
print(len(vehicle_data))
with open('data/khushiauto_cars.json', 'w', encoding='utf-8') as fp:
    for v in vehicle_data:
        fp.writelines(json.dumps(v) + "\n")

{
    "timestamp": "2020-04-30T17:26:39.553325",
    "dealer": "khushiauto",
    "url": "https://www.khushiauto.ca/cars/used/2016-Nissan-Altima-Sedan-114128",
    "price": 9488,
    "make": "Nissan",
    "model": "Altima",
    "year": 2016,
    "body_style": "Sedan",
    "mileage": 136236,
    "transmission": "Automatic",
    "cylinders": 4,
    "displacement": 2.5,
    "drivetrain": "FWD",
    "ext_color": "Silver",
    "int_color": "Beige",
    "doors": 4,
    "stock_id": "2624",
    "vin": "1N4AL3AP5GN359677",
    "features": [
        "ABS Brakes",
        "Air Conditioning",
        "Alloy Wheels",
        "AM/FM Stereo",
        "Backup Camera",
        "Bluetooth",
        "CD Player",
        "Cruise Control",
        "Cup Holder",
        "Digital Clock",
        "Driver Side Airbag",
        "Entertainment System",
        "Fog Lights",
        "Heated Exterior Mirrors",
        "Keyless Entry",
        "Low Tire Pressure Warning",
        "Passenger Airbag",
        "Power L